# JWT 

## File(Flask3): jwt_auth.py


In [1]:

def token_required(f):
   @wraps(f)
   def decorator(*args, **kwargs):
       token = None
       # Grabs the cookie from request  headers
       cookieString = request.headers.get('Cookie')
       # loads the cookie into cookie object
       if cookieString:
           cookie = cookies.SimpleCookie()
           cookie.load(cookieString)
            # if token exist then it grabs the token from the cookie
           if 'token' in cookie:
               token = cookie['token'].value
               print(token)
 
        # if no token exits it shows a message saying valid token is missing 
       if not token:
           return jsonify({'message': 'a valid token is missing'})
        # this code tries to verify the signature of the token by decoding it.
       try:
           data = jwt.decode(token, current_app.config['SECRET_KEY'], algorithms=["HS256"])
           current_user = User.query.filter_by(username=data['name']).first()
        # if signature is not valid or it is not able to decode it writes a message saying token is invalid. 
       except:
           return jsonify({'message': 'token is invalid'})
        # returns current user
       return f(current_user, *args, **kwargs)
       # returns the decorator
   return decorator

from flask import request
@jwt_bp.route('/register', methods=['POST'])
def register():
    # Getting the request data as json
    data = request.get_json()

    # Checking if the required data (username, password) are present
    if not data or not data.get('username') or not data.get('password'):
        # If not, return a JSON response with a message and a 400 status code
        return jsonify({'message': 'Username or password field is empty.'}), 400
    # Querying the User table to check if a user with the provided username already exists
    user = User.query.filter_by(username=data.get('username')).first()

    # If a user is found, return a JSON response with a message and a 400 status code
    if user:
        return jsonify({'message': 'User already exists. Please Log in.'}), 400

    # If a user was not found, the password is hashed using bcrypt
    hashed_password = bcrypt.hashpw(data.get('password').encode('utf-8'), bcrypt.gensalt()).decode('utf-8')  # ensure the hashed password is in string format
    # A new User object is created with the provided username and hashed password
    new_user = User(username=data.get('username'), password=hashed_password)
    # The new User object is added to the db session
    db.session.add(new_user)
    # The db session is committed to save the changes
    db.session.commit()
    # A JSON response is returned with a message and a 201 status code
    return jsonify({'message': 'New user created!'}), 201

@jwt_bp.route('/login', methods=['POST'])
def login():
    # get the username and password from the request
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')

    # authenticate the user, for example by checking the username and password against a database
    user = User.query.filter_by(username=username).first()

    # if the user doesn't exist or the password is wrong, return an error
    if user is None or not user.check_password(password):
        return jsonify({'message': 'Invalid username or password'}), 400

    # if the user is authenticated, create a JWT token for them
    token = jwt.encode(payload= {'name': user.username}, key=current_app.config['SECRET_KEY'], algorithm="HS256")
    print("Token:", token)

    # set the JWT token in a secure HTTP-only cookie in the response
    response = make_response(jsonify({'message': 'Logged in'}), 200)
    response.set_cookie('token', token, secure=False, samesite='Lax', path='/')
    
    print(response.headers)
    return response
    

### Overall Explanation

Verification of JWTs: JWTs are not only created in this file, but are also verified here. The token_required decorator function uses the jwt.decode method to verify the token's signature and extract its payload, ensuring it was issued by this application and hasn't been tampered with. This method provides a secure way of knowing if a request comes from an authenticated user.

Token-based access control: The token_required decorator is used to protect routes that should only be accessible to authenticated users. When applied to a route, it first checks for the presence of a JWT. If the token is missing or invalid, the request is rejected, ensuring that only authenticated users can access protected resources.

Managing session state: The JWT is stored in a cookie, which is set in the user's browser when they log in. This way, the server does not need to maintain session state between requests.

In summary, this file plays a central role in handling the JWT lifecycle, from creation to verification. It also ensures that only authenticated users can access specific routes in the application. 

<br>
<br>
<br>
<br>
<br>
<br>


### Explanation of Code Parts In detail

#### token_required decorator function:

The token_required decorator function is designed to ensure that the user is authenticated before they can access certain routes. Here's what each part does:

The decorator function @wraps(f) preserves information about the decorated function f that might be needed later for introspection.

token = None initializes a token variable as None before trying to extract the token from the cookie.

The code then attempts to extract the token value from the Cookie header in the incoming HTTP request. It first loads the Cookie string into a SimpleCookie object, then checks if a token key exists, if so, it sets the token variable to the token's value.

If no token value could be found, the function returns a JSON response indicating that a valid token is missing.

The function then tries to decode the token using the SECRET_KEY set in the application configuration. If the token is invalid (it can't be decoded, or its signature doesn't match), it returns a JSON response indicating that the token is invalid.

If the token is successfully decoded, the data variable contains the payload of the token, and current_user is set to the user object that corresponds to the username in the payload.

The decorator finally returns the decorated function f, and passes along the current_user and any additional arguments.

#### /register route:

This route allows new users to register. Here's what it does:

It first retrieves the JSON data from the request.

It checks if the username and password fields are present in the request data. If not, it returns a 400 response indicating that the fields are missing.

It then checks if a user with the provided username already exists in the database. If such a user exists, it returns a 400 response indicating that the user already exists.

If the username is not taken, it hashes the provided password using bcrypt, and creates a new User object with the provided username and hashed password.

It then adds the new user to the database and commits the changes.

Finally, it returns a 201 response indicating that the new user was successfully created.

#### /login route:

This route allows existing users to log in. Here's what it does:

It first retrieves the username and password from the JSON data in the request.

It then tries to retrieve the User object that corresponds to the provided username from the database. If no such user exists, or if the provided password doesn't match the stored password for the user, it returns a 400 response indicating that the login credentials are invalid.

If the user is authenticated, it creates a JWT token for them. The token's payload contains the user's username, and it's encoded using the SECRET_KEY set in the application configuration.

It then creates a response indicating that the login was successful, and sets a cookie in the response with the name token and the value of the JWT token.

Finally, it returns the response to the client.





## File(Flask3): User.py

In [2]:
from flask_sqlalchemy import SQLAlchemy
from flask_login import UserMixin
import bcrypt 

db = SQLAlchemy()

# database
class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(20), nullable=False, unique=True)
    password = db.Column(db.String(80), nullable=False)

# The check_password function is a method of the User class. 
    def check_password(self, password):
        return bcrypt.checkpw(password.encode('utf-8'), self.password.encode('utf-8'))

### Explanation

This is very simple and all it does is create rows and outline of the database, the def checkpassword function is used to verify if the provided password matches the hashed password stored in the database.

## File(Flask3): Forms.py

In [ ]:
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField
from wtforms.validators import DataRequired, Length, EqualTo

# Defining a form for registration
class RegisterForm(FlaskForm):
    # A StringField for the username with validators that ensure the field is filled and the length is between 4 and 20 characters
    username = StringField('Username', validators=[DataRequired(), Length(min=4, max=20)])
    # A PasswordField for the password with a validator that ensures the field is filled
    password = PasswordField('Password', validators=[DataRequired()])
    # A PasswordField for the password confirmation with validators that ensure the field is filled and the field is equal to the 'password' field
    confirm_password = PasswordField('Confirm Password', validators=[DataRequired(), EqualTo('password')])
    submit = SubmitField('Sign Up')

# This is for the login
class LoginForm(FlaskForm):
    # A StringField for the username with validators that ensure the field is filled and the length is between 4 and 20 characters
    username = StringField('Username', validators=[DataRequired(), Length(min=4, max=20)])
    # A PasswordField for the password with a validator that ensures the field is filled
    password = PasswordField('Password', validators=[DataRequired()])
    submit = SubmitField('Login')

### Explanation In part of JWT

In the context of JWT authentication with cookie-based login and registration, these forms don't directly handle JWT tokens or cookies. They are primarily concerned with receiving and validating data that a user submits through the login or registration form. Once the form data is validated, the application's authentication logic can use the validated data to generate a JWT token, store it in a cookie, and send it to the client as part of the authentication process.

## File(Flask3): Main.py

In [ ]:
from flask import render_template
from flask_cors import CORS
from flask_migrate import Migrate  # New Import
from flask import render_template, url_for
from nighthawkguessr_api import app, db, project_path
from pathlib import Path
from nighthawkguessr_api.api.todo import todo_bp
from flask import send_from_directory
from nighthawkguessr_api.model.images import initEasyImages

from nighthawkguessr_api.api.leaderboard import leaderboard_bp
from nighthawkguessr_api.model.leaderboards import init_leaderboards
from nighthawkguessr_api.api.leaderboard import leaderboard_bp
from nighthawkguessr_api.api.images import images_bp
from nighthawkguessr_api.api.jwt_auth import jwt_bp
from nighthawkguessr_api.api.pass_api import pass_api, getPassAPI

migrate = Migrate(app, db)  # New Line

app.register_blueprint(todo_bp)
app.register_blueprint(leaderboard_bp)
app.register_blueprint(images_bp)
app.register_blueprint(jwt_bp)
app.register_blueprint(pass_api)

@app.before_first_request
def init_db():
    with app.app_context():
        db.create_all()
        initEasyImages()
        init_leaderboards()

@app.route('/')
def index():
    return render_template('index.html')

# app.add_url_rule('/photos/<path:filename>', endpoint='photos', view_func=app.send_static_file)
# @app.route('/')
# def photo():
#     image_dir = Path.cwd()/"images/easy"
#     images_paths = [i.posix() for i in image_dir.iterdir()]
#     images = [Images("/image/easy/" + image, 250, 250, 1) for image in images_paths]
#     return render_template('photo1.html')

# @app.route('/')
# def capture_image(self):
#     self.cam = cv2.VideoCapture(0)
#     self.img = self.cam.read()
#     self.cam.release()
#     render_template(index.html,ob=self.img)


@app.route('/static/images/easy/<path:path>')
def send_report(path):
    full_filename = project_path.as_posix() + path
    # url_for('static', filename=f'images/easy/{path}')
    return render_template("image_template.html", user_image=path)


if __name__ == "__main__":
    cors = CORS(app, resources={r"*": {"origins": "*"}}, supports_credentials=True)
    app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///./volumes/sqlite.db"
    app.run(debug=True, host="0.0.0.0", port="8200")



### Explanation In Part of JWT

This is the main entry point of my Flask application. I am setting up my application and its routes. The login and register routes are crucial to JWT authentication. These endpoints receive the Information from the frontend, validate the data, and if valid, issue a JWT.

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# Frontend

## File(Nighthawk-guessr): login.html

### Role In JWT

Form Submission and Data Collection: Just like in register.html, the form's submit event is intercepted, the page reload action is prevented, and the entered username and password are collected.

POST Request to the Backend: A POST request is then sent to the '/login' endpoint of the backend API with the username and password.

User Authentication: The backend verifies the provided credentials. If the username exists and the provided password matches the one stored in the database (after being hashed), the user is considered authenticated.

JWT Creation: On successful authentication, the backend generates a JWT that encodes user information. This token is then sent back to the client in the response.

Response Handling: If the authentication is successful, the backend sets the JWT as a cookie in the user's browser. Then, the user is redirected to index.html.

## File(Nighthawk-guessr): register.html

### Role In JWT

Form Submission and Data Collection: When the form is submitted, an event listener attached to the form intercepts the submit event and prevents the default form submission action that reloads the page. It then collects the username and password values entered by the user.

POST Request to the Backend: After the data has been collected, it sends a POST request to the '/register' endpoint of the backend API. The request contains the username and password in the request body.

User Creation: On the backend, a new user record is created with the username and password. The password is typically hashed for security reasons before it's stored in the database.

Response Handling: After the backend processes the request, it sends back a response which is handled in the frontend. If the user registration is successful (i.e., response status is in the 200-299 range), the user is redirected to the login.html page. If an error occurs (e.g., username already exists), an alert message with the error is displayed to the user.

# Overall Baisc Explanation of Flow:

The flow of using JWT for authentication in this application works like this:

User Registration: When a new user tries to register using the register.html page, they fill in their username and password and submit the form. The form data is sent to the /register endpoint in the backend server (main.py) as a POST request, where the user data is processed and stored.

User Login: The user can then login with their username and password on the login.html page. This data is again sent as a POST request, but this time to the /login endpoint in my Flask backend.

In the /login route in main.py, the user's credentials are verified. If they're valid, a new JWT token is created by calling the decoding function in the jwt_auth.py module. 

The newly created JWT token is then sent back to the frontend in the form of a cookie. The set_cookie function from the flask module is used to achieve this. This token cookie will be sent with each subsequent request to the server.

The index.html page checks for the presence of the token cookie on page load. If the cookie doesn't exist, it redirects the user to the login.html page. If the cookie does exist, it means the user has already logged in, and they're allowed to stay on the index.html page.

When the user wants to access protected routes/resources on the site, the server needs to verify their identity. It does this by checking the token cookie that's sent with each request. The token is decoded using the decode_auth_token function in the jwt_auth.py module.

If the token is valid and hasn't expired, the user is granted access to the requested resources. If not, they're given an appropriate error message (unauthorized access, token expired, etc.)



# Issue That I think is happening

I think the issue is something with the CORS, or maybe it running on a http connection, i'm not sure but it is something around that I think.